## Introduction

talk about statistical design of experiements what it is / how it works

## Analysis

### Import Data

Before we begin, let's set up our environment and import the necessary libraries.

If we are using an API key, we need to set uo the environment variable for the API key. 
- Create a .env text file in the root directory of your project and save the key as MPI_KEY=your_api_key_here
- In a Jupyter cell run the following:



In [ ]:
"""
import os
from dotenv import load_dotenv
load_dotenv()
MPI_KEY = os.getenv("MPI_KEY")
"""

**Issues arrived from multiple python versions that kept conflicting with each other  
Before beginning ensure that 3.13.9 and pymatgen 2025.10.7 are running**


In [ ]:
import sys
import pkg_resources

print("Python version:", sys.version)
print("pymatgen version:", pkg_resources.get_distribution("pymatgen").version)

In [ ]:
import os


Create a virtual environment in the terminal 
- python -m venv .venv  

Create a new text file with the name ".gitignore"
- add the text venv/,pycache/ and .env (if used)

Push to GitHub
- git init
- git add . 
- git commit -m "Updated to run in Colab"
- git pull origin main
- git push origin main  

If GitHub is giving merge errors 
- git add -A
- git commit -m "Override remote file with fixed local version that runs in Colab"
- git add MSE1003_Assignment1_RA.ipynb
- git commit -m "Keep local version of notebook"
- git push origin main --force